In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from datetime import datetime
import time
import requests
import pandas as pd
import numpy as np
import json

In [11]:
def generate_glassdoor_url(job_name, page_number):
    base_url = "https://www.glassdoor.co.in/Job/india-"
    job_name = job_name.replace(" ", "-").lower()
    srch = f"IL.0,5_IN115_KO6,{len(job_name)+6}"
    ip = f"_IP{page_number}"
    return (f"{base_url}{job_name}-jobs-SRCH_{srch}{ip}.htm")

In [12]:
def fetch_jobs(url):
    html_content = []
    
    options = Options()
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    driver.get(url)
    driver.minimize_window()
    
    try:
        driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
        time.sleep(2)
    except NoSuchElementException:
        try:
            driver.find_element(By.XPATH, "//*[@id='JAModal']/div/div[2]/span/svg").click()
            time.sleep(2)
        except:
            time.sleep(2)
            pass
    
    done = False
    while not done:
        job_cards = driver.find_elements(By.XPATH,"//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
        for card in job_cards:
            card.click()
            time.sleep(1)

            #Closes the signup prompt
            try:
                driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
                time.sleep(2)
            except NoSuchElementException:
                time.sleep(2)
                pass

            html_content.append(driver.page_source)
            done = True
    
#     time.sleep(100)
    return html_content

In [13]:
# import re
# from collections import namedtuple

# LanguageRegex = re.compile(
#     r"\b("
#     r"ada|algol|assembly|awk|bash|basic|c(\+\+|#)?|cobol|coffeescript|dart|delphi|"
#     r"elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|"
#     r"lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|"
#     r"ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|"
#     r"visual basic"
#     r")\b", re.IGNORECASE)

# FrameworkRegex = re.compile(
#     r"\b("
#     r"angular|aurelia|backbone\.js|bootstrap|bulma|cakephp|cherrypy|django|docker|"
#     r"ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|"
#     r"grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|"
#     r"nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|"
#     r"redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|socket\.io|"
#     r"strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|"
#     r"vue\.js|vuex|wxWidgets|yarn|yesod|zepto"
#     r")\b", re.IGNORECASE)

# DatabaseRegex = re.compile(
#     r"\b("
#     r"cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|"
#     r"redis|sqlite"
#     r")\b", re.IGNORECASE)

# SkillsRegex = re.compile(
#     r"\b("
#     r"amazon web services|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|"
#     r"aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|"
#     r"gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci"
#     r")\b", re.IGNORECASE)

In [14]:
import re
from collections import namedtuple
LanguageRegex = re.compile(r"\b(ada|algol|assembly|awk|bash|c(\+\+|#)?|cobol|css|html5|coffeescript|d|dart|delphi|elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|visual basic)\b", re.IGNORECASE)

FrameworkRegex = re.compile(r"\b(angular|aurelia|backbone\.js|react|spacy|beautifulsoup|bootstrap|bulma|cakephp|cherrypy|django|docker|ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|selenium|socket|socket.io|strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|vue|vuex|wxWidgets|yarn|yesod|zepto)\b", re.IGNORECASE)

DatabaseRegex = re.compile(r"\b(cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|redis|sqlite)\b", re.IGNORECASE)

SkillsRegex= re.compile(r"\b(amazon web services|aws|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci)\b", re.IGNORECASE)

In [15]:
# import re
# from collections import namedtuple

# LanguageRegex = re.compile(
#     r"\b("
#     r"ada|algol|assembly|awk|bash|basic|c(\+\+|#)?|cobol|coffeescript|dart|delphi|"
#     r"elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|"
#     r"lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|"
#     r"ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|"
#     r"visual basic"
#     r")\b", re.IGNORECASE)

# FrameworkRegex = re.compile(
#     r"\b("
#     r"angular|aurelia|backbone\.js|bootstrap|bulma|cakephp|cherrypy|django|docker|"
#     r"ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|"
#     r"grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|"
#     r"nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|"
#     r"redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|socket\.io|"
#     r"strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|"
#     r"vue\.js|vuex|wxWidgets|yarn|yesod|zepto"
#     r")\b", re.IGNORECASE)

# DatabaseRegex = re.compile(
#     r"\b("
#     r"cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|"
#     r"redis|sqlite"
#     r")\b", re.IGNORECASE)

# SkillsRegex = re.compile(
#     r"\b("
#     r"amazon web services|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|"
#     r"aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|"
#     r"gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci"
#     r")\b", re.IGNORECASE)

In [16]:
def process_skills(skills):
    # Get the list of languages
    languages = [lang[0] for lang in skills['languages']]
    
    # Remove empty strings and duplicates, ignoring case
    languages = list(set([lang.strip().lower() for lang in languages if lang.strip()]))
    
    # Combine the results into a dictionary
    return {
        'languages': languages,
        'frameworks': list(set([fr.strip().lower() for fr in skills['frameworks'] if fr.strip()])),
        'databases': list(set([db.strip().lower() for db in skills['databases'] if db.strip()])),
        'skills': list(set([skill.strip().lower() for skill in skills['skills'] if skill.strip()]))
}

In [17]:
import json
import re
from bs4 import BeautifulSoup

def parse_html_to_json(html,count,keyword):
    soup = BeautifulSoup(html, 'html.parser')

    company_name = ""
    job_title = ""
    location = ""
    job_description = ""
    avg_base_pay_est = ""
    company_rating = ""
    company_link = ""
    time_since_posted = ""
    company_skills = "#N/A"

    try:
        company_name = soup.find('div', {'class': 'css-87uc0g e1tk4kwz1'}).text
    except:
        company_name = "#N/A"
    try:
        job_title = soup.find('div', {'class': 'css-1vg6q84 e1tk4kwz4'}).text
    except:
        job_title = "#N/A"
    try:
        location = soup.find('div', {'class': 'css-56kyx5 e1tk4kwz5'}).text
    except:
        location = "#N/A"  
    try:
        avg_base_pay_est = soup.find('div', {'class': 'css-1bluz6i e2u4hf13'}).text
    except:
        avg_base_pay_est = "#N/A"  
    try:
        time_since_posted = soup.find('div', {'class': 'd-flex align-items-end pl-std css-1vfumx3'}).text
    except:
        time_since_posted = "#N/A"    
    try:
        company_rating = soup.find('span', {'class': 'css-1m5m32b e1tk4kwz2'}).text
    except:
        company_rating = "#N/A"

    try:   
        job_description = soup.find('div', {'class': 'css-jrwyhi e856ufb5'}).text
#         job_description = soup.text
    
        language_matches = LanguageRegex.findall(job_description)
        framework_matches = FrameworkRegex.findall(job_description)
        database_matches = DatabaseRegex.findall(job_description)
        skills_matches = SkillsRegex.findall(job_description)

        company_skills = {
            'languages': language_matches,
            'frameworks': framework_matches,
            'databases': database_matches,
            'skills': skills_matches
        }
        company_skills = process_skills(company_skills)  
        ctr=0
        for i in company_skills.values():
            for j in i:
                ctr+=1
        if ctr==0:
            company_skills="#N/A"
        # debug here for 0 skills
    except:
        job_description = "#N/A"
    a_tags = soup.find_all('a', {'class': 'jobLink css-1rd3saf eigr9kq2'}) 
    try:
        company_link="https://glassdoor.co.in"+a_tags[count]["href"]
    except:
        company_link="#N/A"     
        

    # create dictionary object from extracted data
    job_data = {
        'keyword':keyword,
        'company_name': company_name,
        'job_title': job_title,
        'location': location,
        'job_description': job_description,
        'avg_base_pay_est': avg_base_pay_est,
        'company_rating': company_rating,
        'company_link': company_link,
        'time_since_posted': time_since_posted,
        'company_skills': company_skills
    }

    return json.dumps(job_data)


In [18]:
def scrape_job_results(JOB_NAME, NO_OF_PAGES):
    URLS = [generate_glassdoor_url(JOB_NAME, i) for i in range(1,NO_OF_PAGES+1)]

    json_list = []

    for url in URLS:
        html_content = fetch_jobs(url)
        for i in range(30):
            json_list.append(parse_html_to_json(html_content[i], i,JOB_NAME))
            
    return json_list

In [19]:
# ScrapeJobsList = ['Software Engineer', 'Data Scientist', 'Product Manager']

ScrapeJobsList = ['React Developer', 'Data Analyst','Nodejs Developer', 'UI/UX Developer',\
             'Django Developer', 'Flutter Developer', 'Java Developer',\
             'Frontend Developer', 'Full-Stack Developer', 'DevOps Engineer',\
             'Cloud Solutions Architect', 'Machine Learning Engineer',\
             'Artificial Intelligence Specialist', 'Cybersecurity Analyst',\
             'Network Engineer', 'Database Administrator', 'IT Support Specialist',\
             'Python Developer', 'R&D Engineer']

In [20]:
def SCRAPE_ALL_JOB_RESULTS(ScrapeJobsList, NO_OF_PAGES):
    final_json = {}
    for job in ScrapeJobsList:
        final_json[job] = scrape_job_results(job, NO_OF_PAGES)
    return final_json

In [21]:
final_json = SCRAPE_ALL_JOB_RESULTS(ScrapeJobsList, 2)

/tmp/ipykernel_3335/961610879.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=106.0.5249.119)
Stacktrace:
#0 0x55957f4192c3 <unknown>
#1 0x55957f22283a <unknown>
#2 0x55957f2018e3 <unknown>
#3 0x55957f27debd <unknown>
#4 0x55957f291029 <unknown>
#5 0x55957f279d63 <unknown>
#6 0x55957f24e7e3 <unknown>
#7 0x55957f24fa21 <unknown>
#8 0x55957f46718e <unknown>
#9 0x55957f46a622 <unknown>
#10 0x55957f44daae <unknown>
#11 0x55957f46b2a3 <unknown>
#12 0x55957f441ecf <unknown>
#13 0x55957f48b588 <unknown>
#14 0x55957f48b706 <unknown>
#15 0x55957f4a58b2 <unknown>
#16 0x7f79ca749609 <unknown>


In [ ]:
len(final_json)

In [ ]:
final_json

In [ ]:
with open('trying_out_all_20_jobs_response.json', 'w') as f:
    json.dump(final_json, f)

In [31]:
with open('trying_out_all_20_jobs_response.json', 'r') as f:
    data = json.load(f)

In [33]:
data.keys()

dict_keys(['UI/UX Developer', 'Django Developer'])

In [34]:
for i in data.keys():
    for j in range(len(data[i])):
        try:
            print(json.loads(data[i][j]))
        except:
            print("error")
    

{'keyword': 'UI/UX Developer', 'company_name': 'Exodus International Academy', 'job_title': 'IELTS faculty', 'location': 'Pathanāmthitta', 'job_description': "Graduation and IELTS qualified 7bandJob Type: Full-timeSalary: From ₹12,000.00 per monthBenefits:Provident FundSchedule:Day shiftSupplemental pay types:Performance bonusAbility to commute/relocate:Nellimoottil building , Pathanamthitta, Kerala: Reliably commute or planning to relocate before starting work (Required)Education:Master's (Preferred)Experience:Teaching: 1 year (Preferred)Making lesson Plans: 1 year (Preferred)total work: 1 year (Preferred)Language:English (Preferred)", 'avg_base_pay_est': '₹12,000\xa0/mo (est.)', 'company_rating': '#N/A', 'company_link': 'https://glassdoor.co.in/partner/jobListing.htm?pos=101&ao=1110586&s=58&guid=00000187cbb5680aa859d04b991308dd&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_f0aef636&cb=1682749876528&jobListingId=1008567387176&cpc=23F39E5DB52D8DE4&jrtk=3-0-1gv5raq0tihkv801-1gv5raq1aih76800-4eee905

In [35]:
job_array=[]
for i in data.keys():
    for j in range(len(data[i])):

        job_array.append(json.loads(data[i][j]))


In [36]:
count=0
for i in job_array:
    if i["company_link"] =="#N/A":
        count+=1

In [37]:
count

0

In [38]:
jobs=pd.DataFrame(job_array)

In [39]:
jobs

,keyword,company_name,job_title,location,job_description,avg_base_pay_est,company_rating,company_link,time_since_posted,company_skills
0,UI/UX Developer,Exodus International Academy,IELTS faculty,Pathanāmthitta,Graduation and IELTS qualified 7bandJob Type: ...,"₹12,000 /mo (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,25d,N/A
1,UI/UX Developer,freez zone,A/C Technician,Eranakulam,(Amended)Government of IndiaForm GST REG-06[Se...,"₹22,500 /mo (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,25d,N/A
2,UI/UX Developer,DOYD INTERIORS AND CONTRACTORS,SITE CORDINATOR,Eranakulam,SITE CORDINATORCORDINATOR LABOURS WITH WORKSCH...,"₹14,000 /mo (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,25d,N/A
3,UI/UX Developer,Thekkekara's Hot Oven Bakers & Cafe,Head Pastry Chef,Thrissur,Job briefWe are looking for an experienced Hea...,#N/A,#N/A,https://glassdoor.co.in/partner/jobListing.htm...,25d,"{'languages': [], 'frameworks': [], 'databases..."
4,UI/UX Developer,Ginger Technologies,Business Development Executive (BDE),Thrissur,Business Development Executive Responsibilitie...,"₹12,500 /mo (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,25d,N/A
...,...,...,...,...,...,...,...,...,...,...
115,Django Developer,Clovion Tech Solutions Pvt Limited,Python/Django Developer,Nagercoil,We are searching for a Django Developer that i...,"₹12,500 /mo (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,24h,"{'languages': ['python', 'html5', 'javascript'..."
116,Django Developer,Talent Basket3.8,Python/Django Developer,Cochin,Python Full Stack Developerlocation:kochiYear ...,"₹18,00,000 /yr (est.)",3.8,https://glassdoor.co.in/partner/jobListing.htm...,24h,"{'languages': ['python', 'css', 'javascript', ..."
117,Django Developer,Signimus Technologies Private Limited,Django Developer,Remote,Job Title: Django DeveloperJob Summary: We are...,"₹15,000 /mo (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,24h,"{'languages': ['sql', 'html5', 'javascript', '..."
118,Django Developer,Mobile Programming India Pvt Ltd,Software Engineer(Python),Bangalore,"Job briefJob location:- Bangalore,Pune,Gurugra...","₹10,00,000 /yr (est.)",#N/A,https://glassdoor.co.in/partner/jobListing.htm...,24h,"{'languages': ['python', 'html5', 'javascript'..."


In [40]:
len(jobs)

120

In [42]:
for i in jobs["company_skills"]:
    print(i)

N/A
N/A
N/A
{'languages': [], 'frameworks': [], 'databases': [], 'skills': ['chef']}
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
{'languages': [], 'frameworks': [], 'databases': [], 'skills': ['chef']}
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
{'languages': ['python'], 'frameworks': ['django'], 'databases': [], 'skills': []}
{'languages': ['python', 'sql'], 'frameworks': ['django'], 'databases': [], 'skills': []}
{'languages': ['python', 'd'], 'frameworks': ['django', 'docker'], 'databases': ['redis'], 'skills': ['kubernetes', 'prometheus', 'aws', 'nginx', 'redis', 'docker']}
{'languages': ['python'], 'frameworks': ['django'], 'databases': [], 'skills': ['gcp', 'aws']}
{'languages': ['python'], 'frameworks': ['django', 'docker'], 'databases': ['mysql'], 'skills': ['github', 'docker', 'aws', 'nginx']}
{'languages': ['python', 'html5', 'javascript'], 'fram